# V3

- Perform basic read of Jeff Sale Dataset files and read into single Pandas Dataframe
- Attempt to read JSON files, remove emojis in regex, and write back out JSON files
- Use MIKE WINTERS CODE to remove emojis from each file and saving out a new json file

In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import re
import string

from scipy.fftpack import fft, rfft, fftfreq
from scipy.io.wavfile import write
from scipy import log10
from pylab import *

%matplotlib inline

plt.rcParams["figure.figsize"] = (12,9)

In [2]:
def removeEmojis (stringWithEmojis):
    stringNoEmojis = ''.join(filter(lambda x:x in string.printable, stringWithEmojis))
    return stringNoEmojis

In [3]:
# specify data directory
data_dir = 'lyincomey'

In [4]:
os.chdir(data_dir)

In [7]:
files = os.listdir()
files

['lyincomey_18_04_17_08_00_02.txt',
 'lyincomey_18_04_24_03_15_02.txt',
 'lyincomey_18_04_14_14_45_02.txt',
 'lyincomey_18_04_16_11_45_02.txt',
 'lyincomey_18_04_15_12_45_02.txt',
 'lyincomey_18_04_15_00_00_01.txt',
 'lyincomey_18_04_13_07_30_02.txt',
 'lyincomey_18_04_19_18_00_01.txt',
 'lyincomey_18_04_15_01_30_02.txt',
 'lyincomey_18_04_19_19_30_02.txt',
 'lyincomey_18_04_17_04_30_02.txt',
 'lyincomey_18_04_23_03_15_02.txt',
 'lyincomey_18_04_18_12_30_01.txt',
 'lyincomey_18_04_21_06_15_02.txt',
 'lyincomey_18_04_19_23_15_02.txt',
 'lyincomey_18_04_13_19_45_01.txt',
 'lyincomey_18_04_20_00_15_02.txt',
 'lyincomey_18_04_12_14_00_01.txt',
 'lyincomey_18_04_16_16_30_02.txt',
 'lyincomey_18_04_22_19_45_03.txt',
 'lyincomey_18_04_19_00_30_01.txt',
 'lyincomey_18_04_20_14_15_02.txt',
 'lyincomey_18_04_15_18_30_01.txt',
 'lyincomey_18_04_22_11_15_02.txt',
 'lyincomey_18_04_13_00_45_02.txt',
 'lyincomey_18_04_21_12_15_02.txt',
 'lyincomey_18_04_12_19_00_02.txt',
 'lyincomey_18_04_23_17_15_0


### Loop through all files in the directory removing all emojis from the data

In [8]:
# loop through each file in list
for jsonfile in files:
    
    print()
    print()
    print('Reading File: ')
    print(jsonfile)
    print()
    
    # open the json file
    with open(jsonfile) as json_data:
        tweetData = json.load(json_data)
        try:
            tweetList = tweetData['statuses']
        except:
            pass
        
    # loop through each field of the tweetList to removeEmojis
    for idx, status in enumerate(tweetList):

        print(idx)
        tweetList[idx]['text'] = removeEmojis(status['text'])
        tweetList[idx]['user']['description'] = removeEmojis(status['user']['description'])
        tweetList[idx]['user']['name'] = removeEmojis(status['user']['name'])
        tweetList[idx]['user']['location'] = removeEmojis(status['user']['location'])

        if 'url' in status['user']['entities']:
            urls = status['user']['entities']['url']['urls']
            for x in range(len(urls)):
                try:
                    tweetList[idx]['user']['entities']['url']['urls'][x]['display_url'] = removeEmojis(urls[x]['display_url'])
                except:
                    pass


        if 'entities' in status:
            entities = status['entities']
            if 'user_mentions' in entities:
                for x in range(len(status['entities']['user_mentions'])):
                    tweetList[idx]['entities']['user_mentions'][x]['name'] = removeEmojis(status['entities']['user_mentions'][x]['name'])
            if 'urls' in entities:
                urls = status['entities']['urls']
                for x in range(len(urls)):
                    try:
                        tweetList[idx]['entities']['urls'][x]['display_url'] = removeEmojis(urls[x]['display_url'])
                    except:
                        pass


        if 'retweeted_status' in status:
            retweeted_status = status['retweeted_status']
            tweetList[idx]['retweeted_status']['text'] = removeEmojis(retweeted_status['text'])
            tweetList[idx]['retweeted_status']['user']['description'] = removeEmojis(retweeted_status['user']['description'])
            tweetList[idx]['retweeted_status']['user']['name'] = removeEmojis(retweeted_status['user']['name'])

            if 'user_mentions' in retweeted_status['entities']:
                user_mentions = retweeted_status['entities']['user_mentions']
                for x in range(len(user_mentions)):
                    tweetList[idx]['retweeted_status']['entities']['user_mentions'][x]['name'] = removeEmojis(user_mentions[x]['name'])
            # Remove ... from displayed urls
            if 'url' in tweetList[idx]['retweeted_status']['user']['entities']:
                urls = tweetList[idx]['retweeted_status']['user']['entities']['url']['urls']
                for x in range(len(urls)):
                    try:
                        tweetList[idx]['retweeted_status']['user']['entities']['url']['urls'][x]['display_url'] = removeEmojis(urls[x]['display_url'])
                    except:
                        pass

            urls = tweetList[idx]['retweeted_status']['entities']['urls']
            for x in range(len(urls)):
                try:
                    tweetList[idx]['retweeted_status']['entities']['urls'][x]['display_url'] = (removeEmojis(urls[x]['display_url']) + '...')
                except:
                    pass

            if 'quoted_status' in retweeted_status:
                quoted_status = status['retweeted_status']['quoted_status']
                tweetList[idx]['retweeted_status']['quoted_status']['text'] = removeEmojis(quoted_status['text'])
                tweetList[idx]['retweeted_status']['quoted_status']['user']['description'] = removeEmojis(quoted_status['user']['description'])
                tweetList[idx]['retweeted_status']['quoted_status']['user']['name'] = removeEmojis(quoted_status['user']['name'])

                if 'urls' in quoted_status['entities']:
                    urls = quoted_status['entities']['urls']
                    for x in range(len(urls)):
                        try:
                            tweetList[idx]['retweeted_status']['quoted_status']['entities']['urls'][x]['display_url'] = removeEmojis(urls[x]['display_url'])
                        except:
                            pass

        if 'quoted_status' in status:
            quoted_status = status['quoted_status']
            tweetList[idx]['quoted_status']['text'] = removeEmojis(quoted_status['text'])
            tweetList[idx]['quoted_status']['user']['description'] = removeEmojis(quoted_status['user']['description'])
            tweetList[idx]['quoted_status']['user']['name'] = removeEmojis(quoted_status['user']['name'])  

            if 'urls' in quoted_status['entities']:
                urls = quoted_status['entities']['urls']
                for x in range(len(urls)):
                    try:
                        tweetList[idx]['quoted_status']['entities']['urls'][x]['display_url'] = removeEmojis(urls[x]['display_url'])
                    except:
                        pass

    
    ########################
    # end for tweetlist loop
    
    
    # construct new filename
    newFileName = jsonfile + '_dataNoEmojis.json'
    
    # write out new jsonfile
    with open(newFileName, 'w') as f:
        json.dump(tweetList, f, sort_keys=True, indent=4, separators=(',', ': '))
    



Reading File: 
lyincomey_18_04_17_08_00_02.txt

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Reading File: 
lyincomey_18_04_24_03_15_02.txt

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Reading File: 
lyincomey_18_04_14_14_45_02.txt

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
9

64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Reading File: 
lyincomey_18_04_20_22_00_02.txt

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Reading File: 
lyincomey_18_04_17_03_45_02.txt

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Reading File: 
lyincomey_18_04_18_18_45_02.txt

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
5

71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Reading File: 
lyincomey_18_04_18_09_15_01.txt

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96


Reading File: 
lyincomey_18_04_23_18_30_02.txt

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Reading File: 
lyincomey_18_04_17_12_15_01.txt

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
6

82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Reading File: 
lyincomey_18_04_17_09_30_02.txt

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Reading File: 
lyincomey_18_04_22_05_15_01.txt

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Reading File: 
lyincomey_18_04_14_13_30_01.txt

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
7

14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Reading File: 
lyincomey_18_04_17_06_15_02.txt

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Reading File: 
lyincomey_18_04_15_03_15_02.txt

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Reading File: 
lyincomey_18_04_24_00_00_01.txt

0
1
2
3
4
5


16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94


Reading File: 
lyincomey_18_04_23_14_45_01.txt

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


Reading File: 
lyincomey_18_04_13_03_15_01.txt

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95


Reading File: 
lyincomey_18_04_20_08_00_01.txt

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
1

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x87 in position 23: invalid start byte